# eFLINT Tutorial - Voting
_(C) L. Thomas van Binsbergen -- Informatics Institute, University of Amsterdam_  

_The purpose of this document is to informally explain the core features of eFLINT through an example. The worked out specification is not necessarily representative of any real-world policy._

## Fact-type declarations
An eFLINT specification is a collection of type declarations of which the _fact-type_ declarations are used for knowledge representation. The following fact-type declarations introduce the types `citizen`, `candidate` and `administrator`:

In [1]:
Fact citizen
Fact candidate
Fact administrator

Depending on where they occur, identifiers such as `citizen`, `candidate`, and `administrator` can play different roles. For example, when occurring in an expression, an identifier is a placeholder for an instance of the type with the same name. Identifiers can also be used to refer to types themselves and to constructors for creating instances of types, as is shown later. 

In the previous fragment, the declared types are all *atomic*, with its _instances_ coming from the set of (all) strings. To make this explicit, the following type declarations could have been used instead (i.e. are equivalent), indicating that the instances of the types are identified by string literals:

In [2]:
Fact citizen Identified by String
Fact candidate Identified by String
Fact administrator Identified by String

The following code fragments are examples of expressions refering to instances of a (fact-)type: `citizen(Alice)`, `candidate("John")` and `administrator("abc1234")`. String literals are arbitrary sequences of characters between doubles quotes, or are a sequence of alphanumeric characters starting with a capital alphabetical. As such, `"Alice"` and `Alice` are referring to the same string literal. Besides string literals, also integer literals are available (`Identified by Int`).

The instances of a fact-type, *facts*, can be considered as variables to which a truth-value can be assigned. As such, a fact can be considered to hold true, hold false or be undetermined in a given situation. The following _statement_ makes the instance `citizen(Alice)` hold true:

In [3]:
+citizen(Alice)

+citizen("Alice")


To confirm the successful assignment, we can write a *query*, a Boolean expressing following a question mark:

In [4]:
?citizen(Alice).
?citizen(Bob)

Query success
Query failed


The first query succeeded, because of the earlier assignment. The second query failed because the instance `citizen(Bob)` is still undetermined and, by default, fact-types are **closed**, meaning that their instances are implicitly hold false unless otherwise stated (types can also be declared as open types to which the closed-world assumption does not apply, see `eflint3-features/5_open_types`). Note also that in the above code cells, the two (query) statements are seperated by a full stop (`.`).

The statement `-citizen(Alice).` assigns False to the instance `citizen(Alice)` after which it holds false. Alternative, the statement `~citizen(Alice).` can be used to remove the truth assignment to the instance. Because the type `citizen` is closed, the effects of these statements are the same in this case.

Rather than strings or integers, the instances of *composite* types are record-values: tuples with a names for their *fields*, the components of the tuple. The following type declarations instroduce two fact-types whose instances have exactly one field:

In [5]:
Fact voter Identified by citizen
Fact winner Identified by candidate

In both declarations, the *type expression* following `Identified by` consists of a single type name, simultaneously indicating that this name is used as the name for the field and that the field holds instances of the referred type. 

The expression `voter(citizen(Alice))` denotes the instance of `voter` in which the field named `citizen` holds the instance `citizen(Alice)`. The expression `voter(Alice)` is equivalent thanks to implicit coercion of the string `Alice` to an instance of `citizen`. Also equivalent is `voter(citizen=Alice)`; altough in this expression the field is explicitly named. As we shall see later, this form of constructor application makes it possible to write actual parameters in any order (when there are multiple fields) and even omitted.

The intuition behind the fact-type `voter` is that it serves as a predicate to indicate which citizens have voting rights, i.e. `Alice` has voting rights if and only if `voter(Alice)` holds true. That is, the following statement makes Alice a voter:

In [6]:
+voter(Alice).

+voter(citizen("Alice"))


Similarly, `winner` is a predicate over candidates, determining which of the candidates has won the vote, if any.

A type expression of a type declaration referring to multiple types, separated by `*`, determines that the instances of the declared type are records with multiple fields (i.e. `*` is like the *Cartesian product* over two or more types).

In [7]:
Fact vote Identified by citizen * candidate

The fact-types `voter` and `winner` can be seen as describing predicates over citizens and candidates, both identified by strings, respectively. In line with this interpretation, the type `vote` describes a binary relation between citizens and candidates. Instances of `vote` that hold true in a given situation represent proof (e.g. submitted ballots) that the instance's citizen has placed a vote on the instance's candidate.

In [8]:
~voter(Alice)

~voter(citizen("Alice"))


## Expressions

### Variables and placeholders
An identifier `x` appearing in an expression is a variable for the type `x`, i.e. a placeholder for an instance of the type `x`. Additional variables for the type `x` can be introduced by so-called placeholder declarations. For example, in the following fragment, `other-candidate` is introduced as an alias for the identifier `candidate` and is thus a valid variable for the type `candidate`:

In [9]:
Placeholder other-candidate For candidate

Without explicit `Placeholder`-declarations, fact identifiers can be decorated with indices and prime symbols to form variables. For example, `candidate1`, `candidate2` and `candidate'` are all variables for the type  `candidate`. 

The field names of records correspond to a type name, and the fields are instances of that type. For example, the following fact declaration contains a record-type with two different variables of the same type. 

In [10]:
Placeholder proxy For citizen
Fact proxy-of Identified by proxy * citizen

Both fields are instances of the type `citizen`. However, because the field names are different, it is possible to refer the different field values by their name. 

### Projection

The projection operator `<EXPR>.<ID>` makes it possible to refer to field values. If `<EXPR>` evaluates to a record with a field name `<ID>`, then the expression `<EXPR>.<ID>` evaluates to the record's field with name `<ID>`. The static semantics of the language guarantees that `<EXPR>` evaluates to a record with field `<ID>`.

### Constructor application

An identifier plays the role of a constructor (for creating instances) when provided with arguments as part of a constructor-application. For example, the expression `voter(Alice)` applies the identifier `voter` as a constructor to the string `Alice`. That `Alice` is a valid instance of `citizen` is inferred from the type of `voter`.  The same instance is also produced by the expression `voter(citizen(Alice))`. The inner constructor application is an example of a applying an identifier as a constructor to a literal to create an instance of a primitive type. 

An other example of constructor application is `vote(Alice,Chloe)`, constructing an instance representing the potential fact that `Alice`  has voted on `Chloe`. In this example, the relative positioning of the arguments is important. The first argument must be a valid instance of `citizen` and the second argument must be a valid instance of `candidate`, as determined by the type of `vote`. The type of `vote` also determines that exactly two arguments need to be given, as its record-type has two fields.

In another style of constructor application, the arguments are labelled with the field names for which they provide instances. For example, in the expression `voter(candidate = Alice)`, it is made explicit that `Alice` is to take up the role of `candidate` in the created instance for `voter`. There are two benefits to using this style: arguments can be written in any order, and arguments can be omitted. When an argument is omitted, the constructor application is automatically completed with variables for the missing identifiers. For example `vote(candidate = Chloe)` is equivalent to `vote(candidate = Chloe, citizen = citizen)`. Note that both these expressions are only valid if the variable `citizen` is bound by the context in which they appear. Depending on where an expression with unbound variables occurs, the variables are implicitly bound, as discussed later. This style of constructor application is only available for constructors of composite types.

Other expressions are formed by the application of an operator to sub-expressions or consist of a literal such as an string or integer. Boolean and numerical operators are available as they are familiar from Java and other general-purpose languages.

### Quanitifiers and iterating expressions

As suggested before, every occurrence of a variable must be 'bound' by the context in which the variable occurs. Bindings are introduced, for example, by the `Exists` operator, written as `(Exists x1,...,xn : e)`, where `x1,…,xn` are ‘binders’ -- variables for which bindings will be introduced -- and `e` is a Boolean expression. All occurrences of the variables `x1,...,xn` are bound inside `e`. The semantics of `(Exists x1,...,xn : e)` are to determine whether there is a valid combination of instances for the variables `x1,...,xn` so that `e` evaluates to `True`, returning `True` if this is the case and `False` otherwise. The expression `(Exists x1,...,xn : e)` is nondeterministic in the sense that `e` is evaluated an unspecified amount of times and in an unspecified order. However, it is 'effectively deterministic' because the outcome of evaluation is the same regardless (assuming implementations are sound with respect to the given semantics). The `Forall` operator is written similarly and returns True only if all combinators of instances of the variables `x1,...xn` result in `e` evaluating to True. 

When the possible bindings of a variable `x` to instances of its type `t` are enumerated it might be the case that `t` has an unbounded domain (e.g. `String`, representing the domain of all strings). In this case all instances of `t` that hold true in the current situation are enumerated instead. This practical design decision makes it possible to use eFLINT specifications both at runtime (when domains are typically unbounded) and statically with a known or chosen bounded domains. Besides `Exists` and `Forall`, also the `Foreach` operator, introduced below, provides bindings by enumerating instances.

### Instance expressions

An expression evaluating to an instance of some type is referred to as an *instance expression*. By using the `Foreach` operator, written as `(Foreach x1,...,xn: e)`, an instance expression `e` is evaluated multiple times, for all combinators of bindings to `x1,...xn`, collecting all instances produced by these evaluations. As such, an occurrence of `Foreach` yields zero, one or more instances. The `Foreach` operator can be used only as the outermost operator of certain clauses, such as `Derived from`, `Creates` and `Terminates` (introduced below), and as the top-level sub-expression of an aggregator (see below). This design decision ensures the language is 'effectively deterministic'.

The `When` operator, written between its operands as `<EXPR1> When <EXPR2>`, is used to filter out unwanted instances. To evaluate `<EXPR1> When <EXPR2>`, instance `<EXPR1>` and Boolean `<EXPR2>` are evaluated in the same context (with the same active bindings) and gives the single result of `<EXPR1>` but only when `<EXPR2>` evaluates to true. As such, `<EXPR1> When <EXPR2>` evaluates to at most one instance. 

### Aggregators

The `Exists` and `Forall` operators demonstrate a mechanism in which a sub-expression is evaluated repeatedly in different context, with each context binding the binders to a different combination of valid instances of the types of those variables. In the case of `Exists` and `Forall`, the sub-expression must be a Boolean expression. The `Foreach`-operator has the same syntax as `Exists` and `Forall` (except for the keyword name) and expressions formed by the `Foreach`-operator may produce multiple result instances, at most one for each evaluation of its sub-expression. However the `Foreach`-operator can only be used in certain places, for example in combination with an aggregator. 

An aggregator is an operator that combines the possibly many results produced by `Foreach` into a single value. For example, the `Count`-operator is an aggregator that counts the number of evaluation results produced by `Foreach`. The `Sum`-operator expects `Foreach` to produce a collection of numbers, and computes the sum of these numbers. The `Max` and `Min` operators also expect collections of numbers, computing their maximum and minimum respectively.

As an example, the following declaration introduces `number of votes` as a derived fact:

In [11]:
Fact number-of-votes Identified by Int
  Derived from Count(Foreach vote : vote When Holds(vote))

+number-of-votes(0)


In this example, only instances of `vote` that hold true must be counted, as indicated by the occurrence of `Holds(vote)` as the second operand of `When`. In general, the `When` operator has two operands and either evaluates to the result of evaluating the first operand or fails if the second operand evaluates to `False`. Applications of `Foreach`, `Exists` or `Forall` do not consider the results of failing evaluations. Alternative, it can be said that failing evaluations have no result.

## Derivation clauses

### Holds when
As explained earlier, the truth-values assigned to instances can be modified through statements beginning with `+`, `-` or `~`. Later we shall see that events and actions can be defined that also modify truth-assignments when they are triggered. This section explains that the truth of an instance of a type can also be *derived* by one or more derivation rules attached to the type.

An example of type declaration with a derivation rule, in this case specified with a `Holds when` clause, is provided by `has voted`:

In [12]:
Fact has-voted Identified by citizen
  Holds when (Exists candidate : vote(citizen,candidate))

To determine whether an instance `has-voted(<X>)` is derived, the expression `(Exists candidate: vote(citizen,candidate)` is evaluated with the (formal) parameter `citizen` bound to the value provided by the (actual) parameter `<X>`. If the evaluation yields True, then `has-voted(X)` is derived by the derivation rule. 

The following declaration of `has-voted` is equivalent. 

In [13]:
Fact has-voted Identified by citizen
  Holds when vote(citizen, candidate)

This is because every variable appearing in a Boolean expression within a clause of a type, for which holds that the variable is not one of the formal parameters of the type, is implicitly bound by `Exists`. 

In the evaluation of a derivation rule for a primitive type, the name of the type is bound to the instance for which the rule is being evaluated (see also `eflint3-features/4_primitive_holds_when`.

### Derived from

Another form of derivation rule is written as a `Derived from` clause as part of a type declaration. In fact, the `Holds When` clause is syntactic sugar for a particular common usage of `Derived from`. 

#### Holds when conversion for composite types
If `<ID>` is a type name with parameters `<X1...Xn>` and if `<EXPR>` is a Boolean expression, then the clause `Holds When <EXPR>` attached to `<ID>` is equivalent to the `Derived from` clause written as `Derived from <ID>(<X1>,...,<Xn>) When <EXPR>`. The expression written in the clause is an instance expression evaluating to zero, one, or more instances of a type. If the instance expression of a `Derived from` clause does not produce instances of the type to which the clause is attached, then the declaration is ill-typed. All instances computed by the expression of a `Derived from` clause are considered derived by the clause. Parameters are not bound during its evaluation, meaning that in this case, the parameters `<X1...Xn>` are not bound. In any instance expression written as a part of a clause of a type are implicitly bound by an occurrence of `Foreach` (see the section on instance expressions above). 

#### for primitive types
If `<ID>` is a type name of a primitive type and if `<EXPR>` is a Boolean expression, then the clause `Holds When <EXPR>` attached to `<ID>` is equivalent to the `Derived from` clause written as `Derived from <ID> When <EXPR>`.

### Predicates and invariants

A **predicate** is a special fact-type declaration with a derivation rule expressing the condition under which the predicate is considered to hold. Rather than a predicate over some atom, it should be a considered a predicate over the current situation (knowledge base). For example, the following predicate determines whether the vote has concluded (by looking for a winner).

In [14]:
Predicate vote-concluded When (Exists candidate : winner(candidate))
Predicate voters-done    When !(Exists citizen : voter() && !has-voted())

+voters-done()


The instance `vote-concluded()` holds true when the predicate holds.

An **invariant** is a special kind of predicate that is expected to hold true in every possible situation. For example, the following invariant says that there will always be at most one winner.

In [15]:
Invariant at-most-one-winner When 
  Not(Exists winner, winner': winner != winner')

+at-most-one-winner()


The instance `at-most-one-winner()` (see the output above) holds true when the invariant holds. Any invariant that does not hold is reported.

In [16]:
+winner(Chloe). +winner(David). ~winner(Chloe). ~winner(David).

Violated invariant!: at-most-one-winner


## Events

An event-type declaration describes a composite type whose parameters are introduced with an optional `Related to` clause consisting of a comma-separated list of field names. The instances of an event-type are referred to as *events*. An event can be *triggered* and has *effects*. 

### Post-conditions and effects

The following event-type declaration declares an event `fire` that destroys all votes (i.e. truth assignments to instances of the type `vote`). The event has no parameters (no `Related to` clause).

In [17]:
Event fire 
  Obfuscates vote

The `Obfuscates` clause consists of an instance expression `vote`. The variable `vote` is implicitly bound by a `Foreach`. When evaluated, the expression thus evaluates to all instances of the type `vote` (which is determined by the domain and/or the knowledge base, see the section on "instance expressions" above). When the event is triggered, the expression is evaluated and all resulting instances have their truth-assigment removed from the knowledge base. Triggering an event is done by executing a statement consisting of an instance expression evaluating to instances of an event-type (for example `fire().` below).

In [18]:
+vote(Alice,Chloe).
+vote(Bob,David).

~number-of-votes(0)
+number-of-votes(2)
+vote(citizen("Alice"),candidate("Chloe"))
+vote(citizen("Bob"),candidate("David"))


In [19]:
fire().

~number-of-votes(2)
~vote(citizen("Alice"),candidate("Chloe"))
~vote(citizen("Bob"),candidate("David"))
+number-of-votes(0)


Executed transition: fire()


Event-types are used to describe changes to a the current set of facts that are **not** caused by a (known) human actor. For such changes, act-types are used instead (see below).

An `Obfuscates` clause is referred to as a post-condition. Other post-conditions are provided as `Terminates` and `Creates` clauses, modifying the truth-values of the instances their expressions evaluate to to False and True respectively.

## Acts

An act-type declaration is a type declaration that like an event-type declaration describes instances (referred to as *acts* or *actions*) which can be triggered and has effects specified through post-conditions. Like fact-type declarations, an act-type declarations can have derivation clauses. The parameters of an act-type are determined by a `Related to` clause preceded by (optional) `Actor` and `Recipient` clauses. If the `Actor` clause is omitted, a field name `author` (with instances taken from the domain of all strings). The `Recipient` clause is used to describe 'power-liability' relations between the mentioned actors.

In [20]:
Act cast-vote
  Actor citizen
  Recipient candidate
  Holds when voter()
  Conditioned by Not(has-voted())
  Creates vote()

Unlike events, and like duties (see below), actions can be violated. This is the case when an action is triggered which is not enabled (i.e. does not hold true and does not satisfy its conditions).

In [21]:
cast-vote(Johnny,Johnny).

~number-of-votes(0)
+number-of-votes(1)
+vote(citizen("Johnny"),candidate("Johnny"))


Executed transition: cast-vote(citizen("Johnny"),candidate("Johnny"))
Disabled action: cast-vote(citizen("Johnny"),candidate("Johnny"))


In [22]:
-vote(Johnny,Johnny)

~number-of-votes(1)
-vote(citizen("Johnny"),candidate("Johnny"))
+number-of-votes(0)


## Duties

A duty declaration is a type declaration describing the structure of a particular 'duty-claim' relation between two actors: a 'holder' (`Holder`) and a 'claimant' (`Claimant`). The relation may also involve zero or more 'objects' introduced by a `Related to` clause with a list of comma-separated field names. Together, the actors and the objects form the parameters of the duty-type and describe the domain of record-values forming the instances of the type.

If an instance of a duty holds true, then the instance is an element of the duty-claim relation described by the duty (thus stating that the holder of the duty indeed holds the duty towards the claimant). An example is provided by the following fragment:

In [23]:
Duty duty-to-vote Holder citizen Claimant administrator
  Violated when vote-concluded()

The duty-claim relation described by this declaration has no objects. The instances of the `duty-to-vote` type thus has two fields names `citizen` and `administrator`. 

The `Violated when` clauses that can be associated with duty-type declarations determine the conditions under which the duty is considered violated. In this case, the duty is violated when it holds true and when a winner has been announced (i.e. the vote has concluded).

The duty to vote is created by the administrator, when a citizen has been given voting rights.

In [24]:
Act enable-vote
  Actor administrator
  Recipient citizen
  Holds when administrator
  Conditioned by !voter() && !vote-concluded()
  Creates voter(),
          duty-to-vote()

The duty to vote, held by a voter, is satisfied when the voter executes the `cast-vote` action. However, as introduced above, this is not yet specified. To do so, the `Extend` keyword is used.

## Extending declarations

The `Extend` keyword can be used to add additional clauses to an already existing type. The `Extend` keyword is written as the start of a type-extension declaration, followed by either `Fact`, `Act`, `Event`, or `Duty`, indicating which kind of type is extended. The latter is used to syntactically guarantee only valid clauses are written in the remainder of the type-extension declaration. To ensure `cast-vote` removes to duty to vote of a voter, a `Terminates` clause is added to the type.

In [25]:
Extend Act cast-vote
  Terminates duty-to-vote()

## Running and analysing scenarios

#### completing the example
Many of the discussed features come together in the declaration of declare winner given below and completing this example policy description.

In [26]:
Act declare-winner
  Actor administrator
  Recipient candidate
  Holds when administrator && candidate
  Conditioned by Not(vote-concluded()) && voters-done() &&
   (Forall other-candidate : 
      Count(Foreach vote : vote When vote && vote.candidate == other-candidate) < 
      Count(Foreach vote : vote When vote && vote.candidate == candidate)
     When other-candidate != candidate)
  Creates winner(candidate)

The declaration of `declare-winner` determines that an administrator has the power to declare a particular candidate as the winner of the vote if the vote has not been concluded (there is no winner yet), if all of the voters have voted and if all other candidates have (strictly) less votes. The first line starting with `Count` produces the number of voters that have voted for any `other-candidate`. The second line starting with `Count` produces the number of voters that have voted for `candidate`.

### Triggering actions and events

A scenario is written as a sequence of statements, step-by-step changing the knowledge base. A scenario is considered compliant if none of the statements resulted in the violation of an action or a duty.

A scenario typically begins with stating a number facts as axioms, effectively creating an initial state.

In [27]:
+citizen(Alice). ~voter(Alice). +citizen(Bob). +citizen(Chloe). 
+candidate(David). +candidate(Eve).
+administrator(Admin).

+administrator("Admin")
+citizen("Bob")
+citizen("Chloe")
+candidate("David")
+candidate("Eve")
+enable-vote(administrator("Admin"),citizen("Alice"))
+enable-vote(administrator("Admin"),citizen("Bob"))
+enable-vote(administrator("Admin"),citizen("Chloe"))


The following statements express a scenario in which Alice, Bob and Chloe have been giving voting rights by the Admin and Alice votes for Eve and Bob votes for David.

In [28]:
enable-vote(Admin,Alice).
enable-vote(Admin,Bob).
enable-vote(Admin,Chloe).
cast-vote(Alice, Eve).
cast-vote(Bob, David).

~number-of-votes(0)
~voters-done()
~enable-vote(administrator("Admin"),citizen("Alice"))
~enable-vote(administrator("Admin"),citizen("Bob"))
~enable-vote(administrator("Admin"),citizen("Chloe"))
-duty-to-vote(citizen("Alice"),administrator("Admin"))
-duty-to-vote(citizen("Bob"),administrator("Admin"))
+number-of-votes(2)
+has-voted(citizen("Alice"))
+voter(citizen("Alice"))
+vote(citizen("Alice"),candidate("Eve"))
+has-voted(citizen("Bob"))
+voter(citizen("Bob"))
+vote(citizen("Bob"),candidate("David"))
+voter(citizen("Chloe"))
+duty-to-vote(citizen("Chloe"),administrator("Admin"))


Executed transition: enable-vote(administrator("Admin"),citizen("Alice"))
Executed transition: enable-vote(administrator("Admin"),citizen("Bob"))
Executed transition: enable-vote(administrator("Admin"),citizen("Chloe"))
Executed transition: cast-vote(citizen("Alice"),candidate("Eve"))
Executed transition: cast-vote(citizen("Bob"),candidate("David"))


### Running queries

Queries can be used to test the current knowledge base. Queries are written as `?<EXPR>`, where `<EXPR>` is a Boolean expression. The exclamation mark (`!`) in the expression below is logical negation.

In [29]:
?!Enabled(declare-winner(Admin, David)).
?!Enabled(declare-winner(Admin, Eve)).

Query success
Query success


No winner can be declared because both David and Eve have an equal amount of votes. If the scenario continues with Chloe voting for David, we shall see that David can be declared winner, after which the vote has concluded.

In [30]:
cast-vote(Chloe, David).
?Enabled(declare-winner(Admin, David)).
?!Enabled(declare-winner(Admin, Eve)).

~number-of-votes(2)
-duty-to-vote(citizen("Chloe"),administrator("Admin"))
+number-of-votes(3)
+voters-done()
+has-voted(citizen("Chloe"))
+vote(citizen("Chloe"),candidate("David"))


Executed transition: cast-vote(citizen("Chloe"),candidate("David"))
Query success
Query success


In [31]:
declare-winner(Admin, David)

+vote-concluded()
+winner(candidate("David"))


Executed transition: declare-winner(administrator("Admin"),candidate("David"))
